In [1]:
import pandas as pd
import numpy as np
from encode import encode_train_test
from compress import compress_24m
from generate_train_from_24m import generate_train

In [2]:
#首先对训练集和测试集都进行编码
path = '../Data/'
train_data, eval_data = encode_train_test(path+'Handle/sales+search.csv', path+'evaluation_public.csv')

#将每个省，每种车型的数据压缩成一行数据
compress_data = compress_24m(train_data)

#按照时间滑窗12个月生成训练集
train_data = generate_train(compress_data)

#输出训练集
'''train_data.to_csv(path+'Final/train_sales_search_model_1.csv', index=False)'''

range(0, 1320)


"train_data.to_csv(path+'Final/train_sales_search_model_1.csv', index=False)"

In [3]:
base_columns = ['province', 'adcode', 'model', 'bodyType']
fix1 = 'salesVolume'
fix2 = 'popularity'
final_columns = []
for i in range(1, 13):
    final_columns.append(fix1+str(i))
    final_columns.append(fix2+str(i))
    
#计算出2018年1月的数据
change_columns = []
for i in range(13, 25):
    change_columns.append(fix1+str(i))
    change_columns.append(fix2+str(i))
test_data_1 = compress_data[base_columns+change_columns]
test_data_1.columns = base_columns+final_columns

print(test_data_1.head())

  province adcode model bodyType salesVolume1 popularity1 salesVolume2  \
0        3      0     0        3          798         775          532   
0       12      1     0        3          340         561          242   
0        7      2     0        3          129         296           94   
0        2      3     0        3          106         241           63   
0       18      4     0        3          292         250          205   

  popularity2 salesVolume3 popularity3  ... salesVolume8 popularity8  \
0         874          838         610  ...          630         815   
0         539          360         470  ...          362         543   
0         604          160         177  ...          124         224   
0         140          136         157  ...          112         135   
0         249          321         322  ...          316         496   

  salesVolume9 popularity9 salesVolume10 popularity10 salesVolume11  \
0          659         729           544          4